In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Let's understand the data first.**

**Merchandise trade balance:** The Merchandise Trade Balance is the difference in value between imported and exported goods. Data are denominated both in U.S. dollars and renminbi. A positive number indicates a surplus meaning that more goods were exported than imported.
 
**Export:** Value of Exports (money flowing into the Country)
 
**Import:** Value of Imports (money flowing out of the Country)

The goal of this notebook is to analyze US trade's rise and fall with different part of the world.

Looking at the data one can clearly see that in Year **2009** there was a major decline in trade after Trade fell mostly because demand crashed globally and did so particularly in its most import-intensive component – investment.

In **2009**, the U.S. merchandise trade deficit fell by **$308.2 billion (33 percent)** to **$612.4 billion**. U.S. imports and exports fell by **26 percent** and **20 percent**, respectively, in 2009, reflecting the downturn in the U.S. and world economies that resulted in reduced demand for imports and U.S. exports. In 2009, most U.S. merchandise trade deficits with major trading partners also fell.

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings("ignore")
import gc

In [ ]:
imports = pd.read_csv('/kaggle/input/global-patterns-of-us-merchandise-trade/data_imports_of_naics_total_all_merchandise.csv')
imports.shape

In [ ]:
imports.head(3)

In [ ]:
temp_imp = imports
temp_imp['Total_imports'] = temp_imp.sum(axis=1)

temp_imp = temp_imp.melt(id_vars=['Partner','Total_imports'], 
                         var_name="Year", 
                         value_name="Value")

In [ ]:
exports = pd.read_csv('/kaggle/input/global-patterns-of-us-merchandise-trade/data_exports_of_naics_total_all_merchandise.csv')
exports.shape

In [ ]:
exports.head(3)

In [ ]:
temp_exp = exports
temp_exp['Total_exports'] = temp_exp.sum(axis=1)

temp_exp = temp_exp.melt(id_vars=['Partner','Total_exports'], 
                       var_name="Year", 
                       value_name="Value")

In [ ]:
region = pd.read_csv('/kaggle/input/country-mapping-iso-continent-region/continents2.csv')
region = region[['name','region']]

**First Insight:**

**US trade With world trend:**

Lets look at the total import & export of US with world over the years.

In [ ]:
world_imp = temp_imp[temp_imp['Partner'] == 'World']
world_exp = temp_exp[temp_exp['Partner'] == 'World']

world_imp.rename(columns={'Value':'Import'},inplace=True)
world_exp.rename(columns={'Value':'Export'},inplace=True)

world_trade = pd.merge(world_imp,world_exp ,on=['Year','Partner'])


In [ ]:
trace1 = go.Scatter(
    x=world_trade['Year'],
    y=world_trade['Import'],
    name='Import',
    marker=dict(
        color='rgb(34,163,192)'
               )
)
trace2 = go.Scatter(
    x=world_trade['Year'],
    y=world_trade['Export'],
    name='Export'

)

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(trace1)
fig.add_trace(trace2,secondary_y=True)

fig.add_vrect(x0="7.5", x1="6", 
              annotation_text="   decline", annotation_position="top left",
              fillcolor="green", opacity=0.25, line_width=4)

fig['layout'].update(height = 600,title = 'US Import & Export Trend With World (2002-2016)',xaxis=dict(
      tickangle=-90
    ))
fig.update_layout(margin={'b':0,'l':0,'r':0,},
                 paper_bgcolor='rgb(248, 248, 255)',
                 plot_bgcolor='rgb(248, 248, 255)',
                 title={'font':{
                             'family':'monospace',
                             'size':22,
                             'color': 'grey'
                         },
                        'x':0.45,'y':0.9})
fig.show()

**US Top 5 Import partners**

1. Line Plot to plot US import with top 5 Countries

2. Scatter Plot to visualize rise & fall of import with top 5 Countries

3. choropleth Plot to visualize US import With whole world

In [ ]:
imports = imports[1:]

#creating new column with sum of all import over the years
imports['Total_imports'] = imports.sum(axis=1) 

most_imports = imports.nlargest(5,'Total_imports')

In [ ]:
most_imports = most_imports.T.reset_index()
new = most_imports.iloc[0]
most_imports = most_imports[1:-1]
most_imports.columns = new
most_imports.rename(columns={'Partner':'Year'},inplace=True)

In [ ]:
trace1 = go.Scatter(
    x=most_imports['Year'],
    y=most_imports['China'],
    name='China',
    marker=dict(
        color='rgb(34,163,192)'
               )
)
trace2 = go.Scatter(
    x=most_imports['Year'],
    y=most_imports['Canada'],
    name='Canada'

)

trace3 = go.Scatter(
    x=most_imports['Year'],
    y=most_imports['Mexico'],
    name='Mexico'

)

trace4 = go.Scatter(
    x=most_imports['Year'],
    y=most_imports['Japan'],
    name='Japan'

)

trace5 = go.Scatter(
    x=most_imports['Year'],
    y=most_imports['Germany'],
    name='Germany'

)

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(trace1)
fig.add_trace(trace2,secondary_y=True)
fig.add_trace(trace3)
fig.add_trace(trace4)
fig.add_trace(trace5)

fig.add_vline(x=7, line_width=3, line_dash="dash",
              line_color="green",
              annotation_text="In year 2009 US has decline in imports most significantly with Canada",
              annotation_position="top")

fig['layout'].update(height = 600,title = 'US top 5 Import partners (2002-2016)',xaxis=dict(
      tickangle=-90
    ))
fig.update_layout(margin={'b':0,'l':0,'r':0,'t':120},
                 paper_bgcolor='rgb(248, 248, 255)',
                 plot_bgcolor='rgb(248, 248, 255)',
                 title={'font':{
                             'family':'monospace',
                             'size':22,
                             'color': 'grey'
                         },
                        'x':0.45,'y':0.9})
fig.show()

In [ ]:
imp_sca = most_imports.melt(id_vars=['Year'], 
                       var_name="Country", 
                       value_name="Value")
imp_sca['Value'] = imp_sca['Value'].astype(float)
imp_sca['Year'] = imp_sca['Year'].astype(int)

In [ ]:
fig = px.scatter(imp_sca, x="Year", y="Value", animation_frame="Year", animation_group="Country",
           size="Value", color="Country", hover_name="Country",
           log_y=True,size_max=50, range_x=[2002,2016],
           color_discrete_sequence=px.colors.qualitative.Safe)

fig['layout'].update(height = 600,title = 'US top 5 Import partners (2002-2016)',xaxis=dict(
      tickangle=-90
    ))

fig.update_layout(margin={'b':0,'l':0,'r':0,'t':120},
                 paper_bgcolor='rgb(248, 248, 255)',
                 plot_bgcolor='rgb(248, 248, 255)',
                 title={'font':{
                             'family':'monospace',
                             'size':22,
                             'color': 'grey'
                         },
                        'x':0.45,'y':0.9})
fig.show()

In [ ]:
iso_code = pd.read_csv('/kaggle/input/iso-codes/iso_codes.csv')
iso_code = iso_code[['Country','ISO_Code']].drop_duplicates().reset_index(drop=True)

In [ ]:
world_imp = temp_imp[temp_imp['Partner'] != 'World']
world_imp.rename(columns={'Partner':'Country'},inplace=True)

world_import = (
       world_imp.groupby(['Year','Country'])['Value'].agg(['mean'])
      .reset_index()
      .rename(columns={'mean': 'AvgImport'})
      .merge(iso_code,left_on='Country',right_on='Country')
      .sort_values(by=['Year','Country'])
      )

In [ ]:
fig = (
   px.choropleth(
                 world_import               
                ,locations='ISO_Code'               
                ,color='AvgImport'
                ,hover_name='Country'  
                ,hover_data={'ISO_Code':False, 'Year':True,'AvgImport':':.2f'}
                ,animation_frame='Year'   
                ,color_continuous_scale='IceFire' 
                ,height=600)
  .update_layout(
                 title_text='World average US Import dynamics'
                ,title_x=0.3
                ,margin=dict(r=10, t=40, b=10, l=10)
                ,coloraxis_colorbar_title='Import')
 
 .update_layout(margin={'b':0,'l':0,'r':0,'t':120},
                 paper_bgcolor='rgb(248, 248, 255)',
                 plot_bgcolor='rgb(248, 248, 255)',
                 title={'font':{
                             'family':'monospace',
                             'size':22,
                             'color': 'grey'
                         },
                        'x':0.45,'y':0.9})
)
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 800
fig.show()



**US Top 5 Export Partners:**

**Discovery:**

Us Exports are relatively less than Imports.

United States Consumer goods exports are worth US$ 430,334 million, product share of 26.17%. United States Consumer goods imports are worth US$ 949,394 million, product share of 36.98%. United States Capital goods exports are worth US$ 528,561 million, product share of 32.15%.

In [ ]:
most_export = exports[1:]
most_export = most_export.nlargest(5,'Total_exports')

most_export = most_export.T.reset_index()
new = most_export.iloc[0]
most_export = most_export[1:-1]
most_export.columns = new
most_export.rename(columns={'Partner':'Year'},inplace=True)

In [ ]:
trace1 = go.Scatter(
    x=most_export['Year'],
    y=most_export['Canada'],
    name='Canada',
    marker=dict(
        color='rgb(34,163,192)'
               )
)
trace2 = go.Scatter(
    x=most_export['Year'],
    y=most_export['Mexico'],
    name='Mexico'

)

trace3 = go.Scatter(
    x=most_export['Year'],
    y=most_export['China'],
    name='China'

)

trace4 = go.Scatter(
    x=most_export['Year'],
    y=most_export['Japan'],
    name='Japan'

)

trace5 = go.Scatter(
    x=most_export['Year'],
    y=most_export['United Kingdom'],
    name='United Kingdom'

)

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(trace1)
fig.add_trace(trace2,secondary_y=True)
fig.add_trace(trace3)
fig.add_trace(trace4)
fig.add_trace(trace5)

fig.add_vline(x=7, line_width=3, line_dash="dash", line_color="green",annotation_text="Fall of export year 2009",annotation_position="top")

fig['layout'].update(height = 600,title = 'US top 5 EXport partners (2002-2016)',xaxis=dict(
      tickangle=-90
    ))
fig.update_layout(margin={'b':0,'l':0,'r':0,},
                 paper_bgcolor='rgb(248, 248, 255)',
                 plot_bgcolor='rgb(248, 248, 255)',
                 title={'font':{
                             'family':'monospace',
                             'size':22,
                             'color': 'grey'
                         },
                        'x':0.45,'y':0.9})
fig.show()

In [ ]:
exp_sca = most_export.melt(id_vars=['Year'], 
                       var_name="Country", 
                       value_name="Value")
exp_sca['Value'] = exp_sca['Value'].astype(float)
exp_sca['Year'] = exp_sca['Year'].astype(int)

In [ ]:
fig = px.scatter(exp_sca, x="Year", y="Value", animation_frame="Year", animation_group="Country",
           size="Value", color="Country", hover_name="Country",
           log_y=True,size_max=50, range_x=[2002,2016],
           color_discrete_sequence=px.colors.qualitative.Set3)

fig['layout'].update(height = 600,title = 'US top 5 Import partners (2002-2016)',xaxis=dict(
      tickangle=-90
    ))

fig.update_layout(margin={'b':0,'l':0,'r':0,'t':120},
                 paper_bgcolor='rgb(248, 248, 255)',
                 plot_bgcolor='rgb(248, 248, 255)',
                 title={'font':{
                             'family':'monospace',
                             'size':22,
                             'color': 'grey'
                         },
                        'x':0.45,'y':0.9})
fig.show()

In [ ]:
world_exp = temp_exp[temp_exp['Partner'] !='World']
world_exp.rename(columns={'Partner':'Country'},inplace=True)

world_export = (
       world_exp.groupby(['Year','Country'])['Value'].agg(['mean'])
      .reset_index()
      .rename(columns={'mean': 'AvgExport'})
      .merge(iso_code,left_on='Country',right_on='Country')
      .sort_values(by=['Year','Country'])
      )

In [ ]:
fig = (
   px.choropleth(
                 world_export               
                ,locations='ISO_Code'               
                ,color='AvgExport'
                ,hover_name='Country'  
                ,hover_data={'ISO_Code':False, 'Year':True,'AvgExport':':.2f'}
                ,animation_frame='Year'   
                ,color_continuous_scale='mint' 
                ,height=600)
  .update_layout(
                 title_text='World average US Export dynamics'
                ,title_x=0.3
                ,margin=dict(r=10, t=40, b=10, l=10)
                ,coloraxis_colorbar_title='Export')
 
 .update_layout(margin={'b':0,'l':0,'r':0,'t':120},
                 paper_bgcolor='rgb(248, 248, 255)',
                 plot_bgcolor='rgb(248, 248, 255)',
                 title={'font':{
                             'family':'monospace',
                             'size':22,
                             'color': 'grey'
                         },
                        'x':0.45,'y':0.9})
)
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 800
fig.show()


**US trade per region for the entire period**

In [ ]:
region_imp = imports
region_imp = pd.merge(region_imp,region,left_on='Partner',right_on='name')

region_imp = region_imp.groupby('region')['Total_imports'].sum().reset_index().sort_values(by='Total_imports',ascending=False)

region_exp = exports
region_exp = pd.merge(region_exp,region,left_on='Partner',right_on='name')

region_exp = region_exp.groupby('region')['Total_exports'].sum().reset_index().sort_values(by='Total_exports',ascending=False)

In [ ]:
fig = make_subplots(rows=1,cols=2,
                   vertical_spacing=0.25,
                   column_widths=[0.35,0.35],
                   specs=[[{'type':'bar'},{'type':'bar'}]],
                   subplot_titles=['Imports','Exports'])

fig.append_trace(px.bar(region_imp
                      ,y='region'
                      ,x='Total_imports'
                      ,color='Total_imports'
                      ,color_continuous_scale=['orange','red']
                      ,text='Total_imports'
                      ,hover_name='region').data[0],1,1)

fig.append_trace(px.bar(region_exp
                      ,y='region'
                      ,x='Total_exports'
                      ,color='Total_exports'
                      ,text='Total_exports'
                      ,hover_name='region').data[0],1,2)



fig.update_layout(margin={'b':0,'l':0,'r':0,'t':100},
                 paper_bgcolor='rgb(248, 248, 255)',
                 plot_bgcolor='rgb(248, 248, 255)',
                 title={'font':{
                             'family':'monospace',
                             'size':22,
                             'color': 'grey'
                         }, 'text':'US Trade Per Region',
                        'x':0.45,'y':0.9})

fig.show()

In [ ]:
fig = (
    px.line(temp_exp, x="Year", y="Value", color="Partner")
    .update_traces(
        line={"color": "#00CC96", "dash": "dash"},
        mode="lines+markers",
        visible=False,
        name="Export",
    )
    .update_traces(visible=True, selector={"legendgroup": "World"})
    .add_traces(
        px.line(temp_imp, x="Year", y="Value", color="Partner")
        .update_traces(line={"color": "gold",'dash':'dash'}, visible=False,
                       name="Import",
                       mode = 'lines+markers')
        .update_traces(visible=True, selector={"legendgroup": "World"})
        .data
    )
)

fig.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "label": p,
                    "method": "update",
                    "args": [{"visible": [t["legendgroup"] == p for t in fig.data]}],
                }
                for p in temp_imp["Partner"].unique()
            ]
        }
    ]
)
fig.update_layout(margin={'b':0,'l':0,'r':0,'t':100},
                 paper_bgcolor='rgb(248, 248, 255)',
                 plot_bgcolor='rgb(248, 248, 255)',
                 title={'font':{
                             'family':'monospace',
                             'size':22,
                             'color': 'grey'
                         }, 'text':'US Trade With World',
                        'x':0.45,'y':0.9})

fig.show()